# The Algorithm for Solving Systems of Linear Equations (HHL Algorithm)

The HHl algorithm was proposed by Harrow, Hassidim and Lloyd in 2008 (https://arxiv.org/pdf/0811.3171.pdf). It is an algorithm for solving systems of linear equations, which promises an exponential speedup in comparison to classically known algorithms.

The aim is to solve the following equations:
$$ \hat{A} \vec{x} = \vec{b} $$

$\hat{A}$ and $\vec{b}$ are known.

The structure of the algorithm is as can be seen in the next figure.

<img src="hhlcircuitblack.png" />


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aqua.input import get_input_instance
from qiskit_aqua import Operator, run_algorithm, get_eigs_instance, get_reciprocal_instance, get_initial_state_instance
from hhl_func import HHLDEMO
hhl_func = HHLDEMO()

import numpy as np

## Function breakdown

At first we break down the algorithm in its different functions. At the beginning, we have to define the system that we want to solve. That means, the matrix and the vector have to be defined as arrays. 
The needed number of qubits to store the input vector has to be calculated as well. It is the logarithm to the basis 2 of the length of an input vector. For a vector with length 2, we need 1 qubit, length 4, 2 qubits and so on. 

In [4]:
matrix = np.array([[-2, -1], [-1,-2]])
statevec = [1,0]

### State preparation
Then the state $\vec{b}$ has to be prepared. Currently there is no known efficient way to prepare a general given state. We use here the way that is already implemented in Qiskit, after the paper "Synthesis of Quantum Logic Circuits" by Shende, Bullock, Markov (https://arxiv.org/abs/quant-ph/0406176v5).

The state preparation is one of the major drawbacks of the HHL algorithm, as it is assumed that the state is already prepared efficiently. Since this is not yet possible the algorithm is not necessarily faster than a classical one.

In [5]:
hhl_func.getstate("CUSTOM", statevec)

(<qiskit._quantumcircuit.QuantumCircuit at 0x7efca364f160>,
 QuantumRegister(1, 'q0'))

In [6]:
operator = hhl_func.getoperator(matrix)

### Define QPE parameters

The QPE (quantum phase estimation) calculates the eigenvalues of a given matrix. For this the matrix that is to be solved has to be exponentiated. To exponentiate a matrix, that is not diagonal, two different modes can be used in Qiskit to expand the matrix and therefore calculate the approximate exponentiation. The modes are called 'suzuki' and 'trotter' and we have to decide which one to use. 

For both modes the number of time slices has to be defined as well. The more time slices we use the exacter the exponentiation gets. The drawback is the increase in circuit length, which is proportional to the execution time and also increases error rates if executed on a real quantum device.

For the mode 'suzuki' the expansion order has to be set. This is a again a method to better the approximation of the exponentiated matrix. This significantly increases runtime.

At last the accuracy of the eigenvalues has to be defined. The more qubits we use the better the numbers can be represented. Of course a higher number in qubits results in more RAM on the simulator which can at one point exceed the available resources.

As handling matrices with only positive eigenvalues is easier than not positive definite matrices, we need to define if we allow negative eigenvalues.

In [7]:
expansion_mode = 'trotter'
expansion_order = 1
num_time_slices = 30
num_anc = 5
negative_evals = False

In [8]:
hhl_func.construct_qpe_circuit(num_time_slices, num_anc, expansion_mode, expansion_order, negative_evals)

### Define rotation method

At last the rotation method has to be defined. The goal is to map the reciprocal of the eigenvalues stored in the eigenvalue register to the amplitude of the $\vec{b}$ register. This is then proportional to $\frac{1}{\lambda} \vec{b} = \vec{x}$.

For this we need to make a rotation of an ancilla qubit controlled by the eigenvalue register. We have found three different methods to implement this:

__1. Lookup Method:__

The basic idea was to precalculate the rotation values and to map them to the corresponding bit patterns. Furthermore we decided to only include a certain bit length after the most significant bit, which is set to 1, which reduces the number of gates needed. For a complete eigenvalue register length of 8 and a chosen bit length of 5, the maximum difference between the calculated and the real rotation value is about 1 %.

__2. Generated Circuits Method:__

In the paper "Design Automation and Design Space Exploration for Quantum Computers" by M. Soeken et. al (https://arxiv.org/pdf/1612.00631.pdf) they synthesized arithmetic circuits to calculate the reciprocal of a number stored in qubit register by using classical design automation flows and tools. The circuits are optimized for number of qubits. The basic idea was to automate the generation of the long division of a binary number. We are reusing these circuits generated with Verilog and Revs to store the reciprocal in an extra register and afterwards making the controlled rotation.

__3. Long Division:__

The long division method used for the generated circuits can also be implemented directly. This has the advantage that the circuits can be scaled as needed, as for the generated circuit method for every bit size a new file is needed. The resulting precision can also be chosen, since, if the division leaves a remainder, the next division step can be calculated as well, so the resulting register gets longer and more accurate.


In [9]:
reciprocal_mode = "LOOKUP"
hhl_func.construct_reciprocal_circuit(reciprocal_mode)

### Inverse Quantum Phase Estimation

At the end the eigenvalue register has to be unentangled from the ancilla qubit, so the measurement on the ancilla qubit and the resulting amplitude is not dependent on the eigenvalue register. For that the QPE is inverted, which means all gates are inverted and then applied in the inverse order.

In [10]:
inv_qc = hhl_func.construct_inverse_circuit()

### Measurement

For a real application of the HHL the ancilla qubit is measured and if that measurement returns a 1, the register which originally contained $\vec{b}$ now holds $\vec{x}$. This register can then be reused or read out with the help of some operator. The operator is needed to reduce the needed information to only a few features, as otherwise the speedup of the algorithm would be redundant.

For us to test the algorithm the option is to measure the amplitudes of the different states in superposition. This can be either done by executing the circuit very often, or if executed on a simulator, using the state vector option. With this option the exact amplitudes of every state can be read out and the output vector calculated. This is done here. 

The output vector is then compared to the vector which was classically computed. The fidelity gives the agreement between both vectors. If both vectors are the same (excluding a global phase, which has no effect on any measurement), the fidelity is 1.

In [11]:
backend = 'local_qasm_simulator'
ret, circuit = hhl_func.result(backend)
for i in ret.keys():
    print(i, " : ", np.round(ret[i], 4))

no measurements in circuit 'circuit2', classical register will remain all zeros.


probability  :  (0.999+0j)
result  :  [1.e+00-0.j 5.e-04+0.j]
fidelity  :  0.7996
solution  :  [-4.47e-01+0.j -2.00e-04-0.j]
gate_count  :  2478
matrix  :  [[-2 -1]
 [-1 -2]]
invec  :  [1 0]
eigenvalues  :  [-1. -3.]


## Qiskit approach
To make it easier for the users of Qiskit Aqua, every technique for a problem is contained in an algorithm instance. That means that the user does not need to do more than to define the parameters and run the algorithm. It needs a lot less code and understanding of the algorithm.

To make the comparison easier, we use the same parameters here as before.

In [12]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aqua.input import get_input_instance
from qiskit_aqua import Operator, run_algorithm, get_eigs_instance, get_reciprocal_instance, get_initial_state_instance
from hhl_func import HHLDEMO
hhl_func = HHLDEMO()

import numpy as np

In [13]:
params = {
    "algorithm": {
        "name": "HHL",
        "mode": "state_tomography"
    },
    "eigs": {
        "name": "QPE",
        "num_time_slices": num_time_slices,
        "expansion_mode": expansion_mode,
        "expansion_order": expansion_order,
        "negative_evals": True,
        "num_ancillae": num_anc,
    },
    "reciprocal": {
        "name": reciprocal_mode,
        "scale": 1
    },
    "backend": {
        "name": "local_qasm_simulator",
        "shots": 1
    }
}

In [14]:
ret, sv = run_algorithm(params, (matrix, statevec))
for i in ret.keys():
    print(i, " : ", np.round(ret[i], 4))

probability  :  (0.982+0j)
result  :  [-1.    +0.j -0.0091+0.j]
fidelity  :  0.7926
solution  :  [-0.444 +0.j -0.0041+0.j]
gate_count  :  3389
matrix  :  [[-2 -1]
 [-1 -2]]
invec  :  [1 0]
eigenvalues  :  [-1. -3.]
